# Aqua 0.7: Qiskit Chemistry Re-Design Doc

| **Status**        | **Accepted** |
|:------------------|:---------------------------------------------|
| **RFC #**         | 0001                                         |
| **Authors**       | Panos, Donny, Steve, Ivano                   |
| **Submitted**     | 2020-01-06                                   |
| **Updated**       | 2020-01-24                                   |